This is not a difficult exercise, but you need to revise the following technique:
- clean content of a column
- filter column with condition
- create dummy columns from a column which has value of a list
- group by by a column 

In [24]:
import pandas as pd

# Sample DataFrame
data = {'state_id': [1, 1, 2, 2, 3],
        'school_id': [101, 102, 201, 202, 301],
        'subjects': ['engli%sh math chemi%stry', 'math chem%istry', 'engli%sh', 'math en%glish', 'chemistry']}
df = pd.DataFrame(data)


In [25]:
df

,state_id,school_id,subjects
0,1,101,engli%sh math chemi%stry
1,1,102,math chem%istry
2,2,201,engli%sh
3,2,202,math en%glish
4,3,301,chemistry


### 1. Clean the dirty_column to contain only alphanumeric characters

In [26]:
# Clean the dirty_column to contain only alphanumeric characters
df['subjects'] = df['subjects'].str.replace(r'[^a-zA-Z0-9 ]', '', regex=True)

In [27]:
df

,state_id,school_id,subjects
0,1,101,english math chemistry
1,1,102,math chemistry
2,2,201,english
3,2,202,math english
4,3,301,chemistry


### 2. Filter the school with number of subjects > 1

In [28]:
# Define a function to count the number of words in a string
word_count = lambda x: len(x.split())
# Apply the function to the text_column and create a boolean mask
condition = df['subjects'].apply(word_count) > 1
df = df[condition]
df

,state_id,school_id,subjects
0,1,101,english math chemistry
1,1,102,math chemistry
3,2,202,math english


### 3.Split the subjects column into different columns such as each column represent for one subject of shool

In [29]:
# Split the subjects column into separate rows
df['subjects'] = df['subjects'].str.strip().str.split(" ")

/tmp/ipykernel_19496/2339059929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjects'] = df['subjects'].str.strip().str.split(" ")


In [30]:
df

,state_id,school_id,subjects
0,1,101,"[english, math, chemistry]"
1,1,102,"[math, chemistry]"
3,2,202,"[math, english]"


In [33]:
list(df['subjects'])

[['english', 'math', 'chemistry'], ['math', 'chemistry'], ['math', 'english']]

In [38]:
# Create set of unique value in subject
set_subject = set()
for row in list(df['subjects']):
    set_subject = set_subject.union(set(row))

In [39]:
set_subject

{'chemistry', 'english', 'math'}

In [40]:
# Create new column with fixed value = 0
for subject in set_subject:
    df[subject] = 0

/tmp/ipykernel_19496/3116943484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[subject] = 0
/tmp/ipykernel_19496/3116943484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[subject] = 0
/tmp/ipykernel_19496/3116943484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [41]:
df

,state_id,school_id,subjects,english,chemistry,math
0,1,101,"[english, math, chemistry]",0,0,0
1,1,102,"[math, chemistry]",0,0,0
3,2,202,"[math, english]",0,0,0


In [50]:
# Create a function to map value in subjects to each column rpw by row
def dummy_subject(row, subject):
    if subject in row["subjects"]:
        return 1
    else:
        return 0

In [53]:
# Apply mapping binary value from column subjects to other column subject row by row
for subject in set_subject:
    df[subject] = df.apply(dummy_subject, args=(subject,), axis=1)

/tmp/ipykernel_19496/1849048002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[subject] = df.apply(dummy_subject, args=(subject,), axis=1)


In [54]:
df

,state_id,school_id,subjects,english,chemistry,math
0,1,101,"[english, math, chemistry]",1,1,1
1,1,102,"[math, chemistry]",0,1,1
3,2,202,"[math, english]",1,0,1


In [11]:
# Drop the original 'subjects' column
df.drop('subjects', axis=1, inplace=True)
df

/tmp/ipykernel_19496/1610880462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('subjects', axis=1, inplace=True)


,state_id,school_id,chemistry,english,math
0,1,101,1,1,1
1,1,102,1,0,1
3,2,202,0,1,1


### 4. Group by state_id to see the subject of each state

In [12]:
df_state = df.drop('school_id', axis=1)
df_state

Solution 1

In [13]:
sum_by_state = df_state.groupby('state_id').sum().reset_index()

In [14]:
sum_by_state

,state_id,chemistry,english,math
0,1,2,1,2
1,2,0,1,1


Solution 2

In [18]:
df_group = df_state.groupby("state_id").agg({"chemistry": "sum", "english": "sum", "math": "sum"}).reset_index()

In [19]:
df_group

,state_id,chemistry,english,math
0,1,2,1,2
1,2,0,1,1


In [56]:
import pandas as pd

# Create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Math': [90, 85, 88],
        'Science': [88, 82, 90]}
df = pd.DataFrame(data)

# Melt the DataFrame
melted_df = df.melt(id_vars=['Name'], var_name='Subject', value_name='Score')

print(melted_df)

      Name  Subject  Score
0    Alice     Math     90
1      Bob     Math     85
2  Charlie     Math     88
3    Alice  Science     88
4      Bob  Science     82
5  Charlie  Science     90


In [58]:
# Create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Math': [90, 85, 88],
        'Science': [88, 82, 90]}
df = pd.DataFrame(data)

In [59]:
df

,Name,Math,Science
0,Alice,90,88
1,Bob,85,82
2,Charlie,88,90
